##雲端硬碟設定指向

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls 'drive'

MyDrive  Shareddrives


In [3]:
import os
path = "/content/drive/My Drive/Colab Notebooks/ChTemplateNER/"
os.chdir(path)

In [4]:
!ls

1.DataGenerate_entity_茶葉.ipynb		   data_verify
1.DataGenerate_entity_茶葉v2.ipynb		   modeling_bart.py
1.DataGenerate_relation_三元組線性化_茶葉.ipynb    output_dir
2.Train_entity_茶葉.ipynb			   outputs
2.Train_relation_三元組線性化_茶葉.ipynb	   __pycache__
3.Predict_茶葉.ipynb				   seq2seq_model.py
cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb  seq2seq_utils.py
data_entity					   utils_metrics.py
data_relation


In [5]:
#安裝cuda：
#為了加速訓練過程，需要在 GPU 上運行並行處理。為此安裝了 NVIDIA 9.2 cuda 庫：
#要檢查是否安裝了正確的 cuda 編譯器，請運行：!nvcc --version
# 檢查檔案是否存在
import os
if os.path.isfile("cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb"):
  print("cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb檔案存在")
else:
  print("cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb檔案不存在")
  !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb檔案存在
(Reading database ... 165815 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb ...
Unpacking cuda-repo-ubuntu1604-9-2-local (9.2.88-1) over (9.2.88-1) ...
Setting up cuda-repo-ubuntu1604-9-2-local (9.2.88-1) ...
gpg: can't open '/var/cuda-repo-9–2-local/7fa2af80.pub': No such file or directory
Get:1 file:/var/cuda-repo-9-2-local  InRelease
Ign:1 file:/var/cuda-repo-9-2-local  InRelease
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InR

In [6]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
#安裝為 cuda 9.2 配置的 pytorch
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [10]:
#返回当前设备索引
import torch
torch.cuda.current_device()

0

In [11]:
#返回GPU的数量
torch.cuda.device_count()

1

In [12]:
#返回gpu名字，设备索引默认从0开始
torch.cuda.get_device_name(0)

'Tesla T4'

In [13]:
#cuda是否可用
torch.cuda.is_available()

True

In [14]:
import pandas as pd
import logging
from seq2seq_model import Seq2SeqModel
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [15]:
train_data = pd.read_csv("data_entity/tea/train.csv", sep=',').values.tolist()
train_df = pd.DataFrame(train_data, columns=["input_text", "target_text"])
print(train_df)

                                            input_text  \
0     86年開始，茶業改良場凍頂工作站成立了農藥殘留檢測站，抽檢茶菁，民國95年開始改成全部檢驗茶乾。   
1     86年開始，茶業改良場凍頂工作站成立了農藥殘留檢測站，抽檢茶菁，民國95年開始改成全部檢驗茶乾。   
2                                          概述：於108年命名。   
3    ，以系統管理的角度而言稱為整合管理，若就生物科學之角度而言，則稱為健康管理，而二者之間最大的...   
4    ，以系統管理的角度而言稱為整合管理，若就生物科學之角度而言，則稱為健康管理，而二者之間最大的...   
..                                                 ...   
780  。品系2027的親本為臺農8號（母本）與硬枝紅心（父本），命名後成為臺茶12號（金萱）；品系...   
781  。品系2027的親本為臺農8號（母本）與硬枝紅心（父本），命名後成為臺茶12號（金萱）；品系...   
782  。品系2027的親本為臺農8號（母本）與硬枝紅心（父本），命名後成為臺茶12號（金萱）；品系...   
783  。品系2027的親本為臺農8號（母本）與硬枝紅心（父本），命名後成為臺茶12號（金萱）；品系...   
784  。品系2027的親本為臺農8號（母本）與硬枝紅心（父本），命名後成為臺茶12號（金萱）；品系...   

                     target_text  
0    茶業改良場凍頂工作站成立了農藥殘留檢測站是一個事件實體  
1         民國95年開始改成全部檢驗茶乾是一個事件實體  
2                    108年是一個時間實體  
3                  健康管理是一個工作項目實體  
4                  健康管理是一個工作項目實體  
..                           ...  
780                 硬枝紅心是一個茶樹名實體  
781                臺茶12

In [16]:
eval_data = pd.read_csv("data_entity/tea/dev.csv", sep=',').values.tolist()
eval_df = pd.DataFrame(eval_data, columns=["input_text", "target_text"])
print(eval_df)

                                            input_text       target_text
0    製茶類：部分發酵茶類（包種茶、高山茶）六、臺茶22號（沁玉）親本：母本為臺茶12號，父本為青...        包種茶是一個茶名實體
1    製茶類：部分發酵茶類（包種茶、高山茶）六、臺茶22號（沁玉）親本：母本為臺茶12號，父本為青...        高山茶是一個茶名實體
2    製茶類：部分發酵茶類（包種茶、高山茶）六、臺茶22號（沁玉）親本：母本為臺茶12號，父本為青...     臺茶22號是一個茶樹名實體
3    製茶類：部分發酵茶類（包種茶、高山茶）六、臺茶22號（沁玉）親本：母本為臺茶12號，父本為青...        沁玉是一個茶樹名實體
4    製茶類：部分發酵茶類（包種茶、高山茶）六、臺茶22號（沁玉）親本：母本為臺茶12號，父本為青...     臺茶12號是一個茶樹名實體
..                                                 ...               ...
206  夏秋季節非常適合製作紅烏龍茶，紅烏龍茶係結合烏龍茶與紅茶之加工特點與品質特色所新創製出來的特...         紅茶是一個茶名實體
207  ：TetrancychuskanzawaiKishida形態成?：雌?體長約0.44公釐，鮮...  體長約0.44公釐是一個描述實體
208  ：TetrancychuskanzawaiKishida形態成?：雌?體長約0.44公釐，鮮...    鮮紅色或深紅色是一個描述實體
209  ：TetrancychuskanzawaiKishida形態成?：雌?體長約0.44公釐，鮮...    橢圓形或卵圓形是一個描述實體
210  ：TetrancychuskanzawaiKishida形態成?：雌?體長約0.44公釐，鮮...       具足四對是一個描述實體

[211 rows x 2 columns]


In [17]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 5,
    "num_train_epochs": 10,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_during_training": True,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 512,
    "manual_seed": 4,
    "save_steps": 11898,
    "gradient_accumulation_steps": 1,
    "output_dir": "output_dir",
}

In [18]:
# from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline
# tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
# model = BartForConditionalGeneration.from_pretrained("fnlp/bart-base-chinese")
# text2text_generator = Text2TextGenerationPipeline(model, tokenizer)  
# text2text_generator("北京是[MASK]的首都", max_length=50, do_sample=False)

In [19]:
# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="fnlp/bart-base-chinese",
    args=model_args,
    # use_cuda=False,
)

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [20]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

INFO:seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/785 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:seq2seq_model: Training started


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/157 [00:00<?, ?it/s]

...

INFO:seq2seq_model:{'eval_loss': 0.3915628178252114, 'eval_acc': 0.9127218934911243}


Running Epoch 9 of 10:   0%|          | 0/157 [00:00<?, ?it/s]

INFO:seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/211 [00:00<?, ?it/s]

INFO:seq2seq_model:{'eval_loss': 0.39511107018700353, 'eval_acc': 0.9127218934911243}
INFO:seq2seq_model:Saving model into output_dir
INFO:seq2seq_model: Training of fnlp/bart-base-chinese model complete. Saved to output_dir.


In [21]:
# Evaluate the model
results = model.eval_model(eval_df)

INFO:seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/211 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/27 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/27 [00:00<?, ?it/s]

INFO:seq2seq_model:{'eval_loss': 0.39511107018700353, 'eval_acc': 0.9127218934911243}


In [22]:
# Use the model for prediction

print(model.predict(["碧螺春綠茶為不發酵茶，在臺茶風味輪的歸納中隸屬於臺灣綠茶，傳統綠茶製造過程不經過萎凋及攪拌的程序，其製程為：茶菁→殺菁→揉捻→初乾→解塊→乾燥，而碧螺春雖為炒菁綠茶，但農友在生產時常會攤放靜置茶菁數小時，以改善茶葉的風味。新北市三峽茶區毗鄰新店、土城、樹林、鶯歌、大溪，連接文山茶區，是臺灣唯一僅剩的專業炒菁綠茶產區，茶樹品種主要為青心柑仔及青心烏龍，其種植地區白雞山山峰環繞，雲霧濛密，氣候涼爽，土質良好。三峽地區農友開墾種茶歷史已逾200年，在清明節前當葉片生長至一心三葉時，用手採一心二葉之嫩芽製成的碧螺春綠茶，它的形、色、香、味最佳。碧螺春外觀新鮮碧綠，芽尖白毫多，形狀細緊捲曲似螺旋，乾茶清香鮮雅、亮麗自然，茶湯碧綠清澈、鮮活爽口，品質獨樹一格。碧螺春的產期在每年 3 月至 12 月，其中在 3~5 月及 10~12 月生產的品質較佳。"]))
print(model.predict(["其製程為：茶菁→殺菁→揉捻→初乾→解塊→乾燥，而碧螺春雖為炒菁綠茶，但農友在生產時常會攤放靜置茶菁數小時，以改善茶葉的風味"]))
print(model.predict(["碧螺春的產期在每年 3 月至 12 月，其中在 3~5 月及 10~12 月生產的品質較佳。"]))
print(model.predict(["紅茶為全發酵茶。茶葉經長時間室內萎凋，再經揉捻過程使葉片細胞劇烈破壞，促使茶多酚與氧化酵素進行反應，最後補足發酵，令葉底完全變紅。臺灣紅茶以大葉種品種製成為主，例如臺茶 18 號（紅玉），色澤墨黑油潤泛紫光，湯色金紅，具有淡淡的肉桂味與薄荷香，滋味甘濃甜爽；其後推廣的臺茶 21 號（紅韻），則具芸香科花香，兩者共同譜出臺灣高香紅茶新形象。近年來其他茶區亦利用夏秋季小葉種茶菁原料產製小葉種紅茶，或是使用受茶小綠葉蟬刺吸（著蜒）的幼嫩茶芽，製成蜜香紅茶，皆受到許多消費者喜愛。紅茶外觀色澤宜烏黑油潤泛紫光，條索緊結勻齊，具金黃白毫者為佳（臺茶 18 號除外）；湯色紅豔、澄清明亮泛油光；不同品種所產製的紅茶香氣各具特色，但皆以清純濃郁者為佳；滋味醇和、濃強鮮爽；葉底肥軟鮮活且紅勻明亮。"]))
print(model.predict(["竹山鎮山坪頂茶區早期推廣以臺茶 13 號、臺茶 12 號為主，因其產量高、管理容易、產製技術純熟，且其部份茶園所產製之茶菁，外售至山下大型茶工廠製造，茶農收益穩定，且其屬山坡地，更新不易，至今仍保持以該品種為大宗。茶菁採摘以手採為主，產製茶類為半球形部分發酵茶，因其海拔約略 400-600 公尺，故其製茶發酵度較高，茶湯滋味濃稠度較足。夏季亦利用臺茶 12 號等茶菁產製小葉種紅茶，紅茶成品茶湯滋味濃稠感佳。"]))

['碧 螺 春 綠 茶 是 一 個 茶 名 實 體']
['初 乾 是 一 個 工 作 項 目 實 體']
['碧 螺 春 是 一 個 茶 樹 名 實 體']
['紅 茶 是 一 個 茶 名 實 體']
['紅 茶 是 一 個 茶 名 實 體']


In [23]:
from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
model = BartForConditionalGeneration.from_pretrained('outputs/best_model')
text2text_generator = Text2TextGenerationPipeline(model, tokenizer)
answer=text2text_generator("螯蝦瘟（Crayfish plague）為造成淡水螯蝦大量死亡之真菌性傳染病，病 原為一種水中黴菌 Aphanomyces astaci，藉由在水中散佈黴菌孢子，造成淡 水螯蝦大量死亡，死亡率可高達 100%，過去由北美螯蝦帶原傳入歐洲造成歐 洲原生螯蝦大量死亡， 2013 年底我國養殖澳洲螯蝦 （Cherax quadricarinatus） （又名淡水小龍蝦、紅螯螯蝦）首次發生螯蝦瘟疫情，感染場螯蝦大量死亡， 發病率約 72%，死亡率約 66%。", max_length=50, do_sample=False) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [24]:
print(answer[0]["generated_text"])

螯 蝦 瘟 是 一 個 蟲 實 體
